In [36]:
import pandas as pd
import numpy as np
veh_data = pd.read_csv('data/citibike-tripdata.csv', sep=',')
display(veh_data.head())



,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,2018-09-01 00:00:05.2690,2018-09-01 00:27:20.6340,252.0,MacDougal St & Washington Sq,40.732264,-73.998522,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,25577,Subscriber,1980,1
1,2018-09-01 00:00:11.2810,2018-09-01 00:02:23.4810,314.0,Cadman Plaza West & Montague St,40.693830,-73.990539,3242.0,Schermerhorn St & Court St,40.691029,-73.991834,34377,Subscriber,1969,0
2,2018-09-01 00:00:20.6490,2018-09-01 00:55:58.5470,3142.0,1 Ave & E 62 St,40.761227,-73.960940,3384.0,Smith St & 3 St,40.678724,-73.995991,30496,Subscriber,1975,1
3,2018-09-01 00:00:21.7460,2018-09-01 00:07:38.5830,308.0,St James Pl & Oliver St,40.713079,-73.998512,3690.0,Park Pl & Church St,40.713342,-74.009355,28866,Subscriber,1984,2
4,2018-09-01 00:00:27.3150,2018-09-01 02:21:25.3080,345.0,W 13 St & 6 Ave,40.736494,-73.997044,380.0,W 4 St & 7 Ave S,40.734011,-74.002939,20943,Customer,1994,1


In [33]:
vel_df = veh_data.drop(['start station id', 'end station id'], axis=1)
vel_df['age'] = 2018 - vel_df['birth year']
vel_df.drop(['birth year'], axis=1)
display(vel_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station name       299831 non-null  object 
 3   start station latitude   300000 non-null  float64
 4   start station longitude  300000 non-null  float64
 5   end station name         299831 non-null  object 
 6   end station latitude     300000 non-null  float64
 7   end station longitude    300000 non-null  float64
 8   bikeid                   300000 non-null  int64  
 9   usertype                 300000 non-null  object 
 10  birth year               300000 non-null  int64  
 11  gender                   300000 non-null  int64  
 12  age                      300000 non-null  int64  
dtypes: float64(4), int64(4), object(5)
memory usage: 29.8+ MB


None

In [39]:
display(vel_df[vel_df['age'] > 60].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11837 entries, 53 to 299990
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype          
---  ------                   --------------  -----          
 0   starttime                11837 non-null  datetime64[ns] 
 1   stoptime                 11837 non-null  datetime64[ns] 
 2   start station name       11837 non-null  object         
 3   start station latitude   11837 non-null  float64        
 4   start station longitude  11837 non-null  float64        
 5   end station name         11837 non-null  object         
 6   end station latitude     11837 non-null  float64        
 7   end station longitude    11837 non-null  float64        
 8   bikeid                   11837 non-null  int64          
 9   usertype                 11837 non-null  object         
 10  birth year               11837 non-null  int64          
 11  gender                   11837 non-null  int64          
 12  age             

None

In [47]:
vel_df['stoptime'] = pd.to_datetime(vel_df['stoptime'])
vel_df['starttime'] = pd.to_datetime(vel_df['starttime'])
vel_df['trip duration'] = (vel_df['stoptime'] - vel_df['starttime'])
display(vel_df['trip duration'].mean().seconds)


1126

In [51]:
vel_df['weekend'] = vel_df['starttime'].dt.dayofweek.apply(lambda x: 1 if x in [5, 6] else 0)
display(vel_df['weekend'].value_counts())

0    184865
1    115135
Name: weekend, dtype: int64

In [70]:
import datetime as dt
def timeofday(timedata):
    td_1 = 0
    td_2 = 6
    td_3 = 12
    td_4 = 18
    result = ' '
    if td_1 <= timedata <= td_2: result = 'night'
    if td_2 < timedata <= td_3: result = 'morning'
    if td_3 < timedata <= td_4: result = 'day'
    if td_4 < timedata <= 23: result = 'evening'
    return result

vel_df['time of day'] = vel_df['starttime'].dt.hour.apply(timeofday) 
display(vel_df['time of day'].value_counts())
display(vel_df[vel_df['time of day'] == 'day'].shape[0] / vel_df[vel_df['time of day'] == 'night'].shape[0])

day        143012
morning     95530
evening     46373
night       15085
Name: time of day, dtype: int64

9.480411004308916

In [69]:
import datetime as dt
def timeofday(timedata):
    td_1 = dt.time(0, 0, 0, 0)
    td_2 = dt.time(6, 0, 0, 0)
    td_3 = dt.time(12, 0, 0, 0)
    td_4 = dt.time(18, 0, 0, 0)
    result = ' '
    if td_1 < timedata < td_2: result = 'night'
    if td_2 < timedata < td_3: result = 'morning'
    if td_3 < timedata < td_4: result = 'day'
    if td_4 < timedata < dt.time(23, 59, 59, 0): result = 'evening'
    return result

vel_df['time of day'] = vel_df['starttime'].dt.time.apply(timeofday) 
display(vel_df['time of day'].value_counts())
display(vel_df[vel_df['time of day'] == 'day'].shape[0] / vel_df[vel_df['time of day'] == 'night'].shape[0])

day        133070
morning     81698
evening     75472
night        9760
Name: time of day, dtype: int64

13.63422131147541